In [23]:
from itertools import combinations
import networkx as nx
import matplotlib.pyplot as plt
import random
import math

In [24]:
def is_graph_minimum_vertex_cover(G, vertex_cover):
    for u in G:
        for v in G[u]:
            if u not in vertex_cover and v not in vertex_cover:
                return False
    return True
    
def brute_force(G):
    vertices = list(G.nodes())
    n = len(vertices)
    
    for r in range(1, n+1):
        for subset in combinations(vertices, r):
            vertex_cover = set(subset)
            if is_graph_minimum_vertex_cover(G, vertex_cover):
                return vertex_cover, len(vertex_cover)
    
    return None, None

In [22]:
def initial_solution(graph):
    num_nodes = len(graph.nodes())
    
    solution = [random.random() < 0.25 for _ in range(num_nodes)]
    
    solution_set = {i for i, included in enumerate(solution) if included}
    
    if not is_valid_cover(solution_set, graph):
        uncovered_edges = [(u, v) for u, v in graph.edges() if u not in solution_set and v not in solution_set]
        
        for u, v in uncovered_edges:
            if u not in solution_set:
                solution_set.add(u)
            elif v not in solution_set:
                solution_set.add(v)
    
    return solution_set

def calculate_cost(vertex_cover, graph):
    return len(vertex_cover)

def is_valid_cover(vertex_cover, graph):
    for u, v in graph.edges():
        if u not in vertex_cover and v not in vertex_cover:
            return False
    return True

def get_neighbor(vertex_cover, graph):
    neighbor = vertex_cover.copy()

    if len(vertex_cover) > 1 and random.random() > 0.5:
        vertex_to_remove = random.choice(list(neighbor))
        neighbor.remove(vertex_to_remove)
        
        if not is_valid_cover(neighbor, graph):
            neighbor.add(vertex_to_remove)  
    else:
        uncovered_edges = set(graph.edges()) - set((u, v) for u in neighbor for v in neighbor if graph.has_edge(u, v))
        if uncovered_edges:
            vertex_to_add = random.choice(list(uncovered_edges))[0]
            neighbor.add(vertex_to_add)

    return neighbor

def simulated_annealing(graph, max_iter):
    current_solution = initial_solution(graph)
    current_cost = calculate_cost(current_solution, graph)
    best_solution = current_solution
    best_cost = current_cost

    for i in range(1, max_iter):
        neighbor = get_neighbor(current_solution, graph)
        neighbor_cost = calculate_cost(neighbor, graph)
            
        if neighbor_cost < current_cost:
            current_cost = neighbor_cost
            current_solution = neighbor 
            if neighbor_cost < best_cost:
                best_solution = neighbor
                best_cost = neighbor_cost
        else:
            p = 1 / i ** 0.5
            q = random.random()
            if q < p:
                current_cost = neighbor_cost
                current_solution = neighbor

    return best_solution, best_cost